# 第6章 モデルのチューニング
## 6.1 パラメータチューニング
### 6.1.4 ベイズ最適化

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

cancer=load_breast_cancer()
X=pd.DataFrame(cancer.data)
y=pd.DataFrame(cancer.target)

In [2]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y=train_test_split(X,y)

# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [3]:
# xgboostによる学習・予測を行うクラス
import xgboost as xgb

class Model:
    
    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

In [6]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss

def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}

In [7]:
# 探索するパラメータの空間を指定する
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}

In [23]:
# hyperoptによるパラメータ探索の実行
# 探索回数（100回程度で十分）
max_evals = 10  #　ここでは10で試す
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

[23:58:46] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-error:0.06270	eval-error:0.05607                                                                             

[1]	train-error:0.05329	eval-error:0.06542                                                                             

[2]	train-error:0.05329	eval-error:0.06542                                                                             

[3]	train-error:0.05329	eval-error:0.06542                                                                             

[4]	train-error:0.03135	eval-error:0.06542                                                                        

[3]	train-error:0.03135	eval-error:0.05607                                                                             

[4]	train-error:0.03762	eval-error:0.05607                                                                             

[5]	train-error:0.02508	eval-error:0.05607                                                                             

[6]	train-error:0.02508	eval-error:0.05607                                                                             

[7]	train-error:0.02194	eval-error:0.05607                                                                             

[8]	train-error:0.01881	eval-error:0.05607                                                                             

[9]	train-error:0.01881	eval-error:0.05607                                                                             

params: {'gamma': 0.2, 'max_depth': 7, 'min_child_weight': 4.0}, logloss: 0.1178                                       
100%|████████████████████████████

{'gamma': 0.30000000000000004, 'max_depth': 4.0, 'min_child_weight': 1.0}

In [26]:
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')

best params:{'gamma': 0.30000000000000004, 'max_depth': 4, 'min_child_weight': 1.0}, score:0.0922


### 6.1.6 ニューラルネットワークとそのチューニング

In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

cancer=load_breast_cancer()
X=pd.DataFrame(cancer.data)
y=pd.DataFrame(cancer.target)

In [4]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y=train_test_split(X,y)

# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [8]:
# -----------------------------------
# ニューラルネットのパラメータチューニングの例
# -----------------------------------
from hyperopt import hp
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import ReLU, PReLU
from keras.layers.core import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from sklearn.preprocessing import StandardScaler

# 基本となるパラメータ
base_param = {
    'input_dropout': 0.0,
    'hidden_layers': 3,
    'hidden_units': 96,
    'hidden_activation': 'relu',
    'hidden_dropout': 0.2,
    'batch_norm': 'before_act',
    'optimizer': {'type': 'adam', 'lr': 0.001},
    'batch_size': 64,
}

# 探索するパラメータの空間を指定する
param_space = {
    'input_dropout': hp.quniform('input_dropout', 0, 0.2, 0.05),
    'hidden_layers': hp.quniform('hidden_layers', 2, 4, 1),
    'hidden_units': hp.quniform('hidden_units', 32, 256, 32),
    'hidden_activation': hp.choice('hidden_activation', ['prelu', 'relu']),
    'hidden_dropout': hp.quniform('hidden_dropout', 0, 0.3, 0.05),
    'batch_norm': hp.choice('batch_norm', ['before_act', 'no']),
    'optimizer': hp.choice('optimizer',
                           [{'type': 'adam',
                             'lr': hp.loguniform('adam_lr', np.log(0.00001), np.log(0.01))},
                            {'type': 'sgd',
                             'lr': hp.loguniform('sgd_lr', np.log(0.00001), np.log(0.01))}]),
    'batch_size': hp.quniform('batch_size', 32, 128, 32),
}


class MLP:

    def __init__(self, params):
        self.params = params
        self.scaler = None
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        # パラメータ
        input_dropout = self.params['input_dropout']
        hidden_layers = int(self.params['hidden_layers'])
        hidden_units = int(self.params['hidden_units'])
        hidden_activation = self.params['hidden_activation']
        hidden_dropout = self.params['hidden_dropout']
        batch_norm = self.params['batch_norm']
        optimizer_type = self.params['optimizer']['type']
        optimizer_lr = self.params['optimizer']['lr']
        batch_size = int(self.params['batch_size'])

        # 標準化
        self.scaler = StandardScaler()
        tr_x = self.scaler.fit_transform(tr_x)
        va_x = self.scaler.transform(va_x)

        self.model = Sequential()

        # 入力層
        self.model.add(Dropout(input_dropout, input_shape=(tr_x.shape[1],)))

        # 中間層
        for i in range(hidden_layers):
            self.model.add(Dense(hidden_units))
            if batch_norm == 'before_act':
                self.model.add(BatchNormalization())
            if hidden_activation == 'prelu':
                self.model.add(PReLU())
            elif hidden_activation == 'relu':
                self.model.add(ReLU())
            else:
                raise NotImplementedError
            self.model.add(Dropout(hidden_dropout))

        # 出力層
        self.model.add(Dense(1, activation='sigmoid'))

        # オプティマイザ
        if optimizer_type == 'sgd':
            optimizer = SGD(lr=optimizer_lr, decay=1e-6, momentum=0.9, nesterov=True)
        elif optimizer_type == 'adam':
            optimizer = Adam(lr=optimizer_lr, beta_1=0.9, beta_2=0.999, decay=0.)
        else:
            raise NotImplementedError

        # 目的関数、評価指標などの設定
        self.model.compile(loss='binary_crossentropy',
                           optimizer=optimizer, metrics=['accuracy'])

        # エポック数、アーリーストッピング
        # あまりepochを大きくすると、小さい学習率のときに終わらないことがあるので注意
        nb_epoch = 200
        patience = 20
        early_stopping = EarlyStopping(patience=patience, restore_best_weights=True)

        # 学習の実行
        history = self.model.fit(tr_x, tr_y,
                                 epochs=nb_epoch,
                                 batch_size=batch_size, verbose=1,
                                 validation_data=(va_x, va_y),
                                 callbacks=[early_stopping])

    def predict(self, x):
        # 予測
        x = self.scaler.transform(x)
        y_pred = self.model.predict(x)
        y_pred = y_pred.flatten()
        return y_pred

In [9]:
# -----------------------------------
# パラメータチューニングの実行

from hyperopt import fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import log_loss


def score(params):
    # パラメータセットを指定したときに最小化すべき関数を指定する
    # モデルのパラメータ探索においては、モデルにパラメータを指定して学習・予測させた場合のスコアとする
    model = MLP(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


# hyperoptによるパラメータ探索の実行
max_evals = 10
trials = Trials()
history = []
fmin(score, param_space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# trialsからも情報が取得できるが、パラメータを取得しにくい
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')

Epoch 1/200                                                                                                            
1/5 [=====>........................]                                                                                   
 - ETA: 0s - loss: 0.8556 - accuracy: 0.3906                                                                           
                                                                                                                      
2/5 [===========>..................]                                                                                   
 - ETA: 0s - loss: 0.8627 - accuracy: 0.4141                                                                           
                                                                                                                      
5/5 [==============================]                                                                                   
 - ETA: 0s - loss: 0.8809 - accuracy: 0.42

5/5 [==============================]                                                                                   
 - ETA: 0s - loss: 0.6979 - accuracy: 0.5737                                                                           
                                                                                                                      
5/5 [==============================]                                                                                   
 - 0s 34ms/step - loss: 0.6979 - accuracy: 0.5737 - val_loss: 0.6100 - val_accuracy: 0.6075                            

Epoch 19/200                                                                                                           
1/5 [=====>........................]                                                                                   
 - ETA: 0s - loss: 0.5827 - accuracy: 0.6250                                                                           
                                        

 - ETA: 0s - loss: 0.4979 - accuracy: 0.7344                                                                           
                                                                                                                      
5/5 [==============================]                                                                                   
 - 0s 28ms/step - loss: 0.5506 - accuracy: 0.6897 - val_loss: 0.5139 - val_accuracy: 0.8037                            

Epoch 37/200                                                                                                           
1/5 [=====>........................]                                                                                   
 - ETA: 0s - loss: 0.5348 - accuracy: 0.7344                                                                           
                                                                                                                      
4/5 [=======================>......]     

                                                                                                                      
5/5 [==============================]                                                                                   
 - 0s 42ms/step - loss: 0.4508 - accuracy: 0.8339 - val_loss: 0.4262 - val_accuracy: 0.9346                            

Epoch 54/200                                                                                                           
1/5 [=====>........................]                                                                                   
 - ETA: 0s - loss: 0.4904 - accuracy: 0.7812                                                                           
                                                                                                                      
5/5 [==============================]                                                                                   
 - ETA: 0s - loss: 0.4494 - accuracy: 0.8

1/5 [=====>........................]                                                                                   
 - ETA: 0s - loss: 0.4078 - accuracy: 0.8438                                                                           
                                                                                                                      
5/5 [==============================]                                                                                   
 - 0s 33ms/step - loss: 0.3854 - accuracy: 0.8589 - val_loss: 0.3579 - val_accuracy: 0.9439                            

Epoch 72/200                                                                                                           
1/5 [=====>........................]                                                                                   
 - ETA: 0s - loss: 0.4147 - accuracy: 0.8750                                                                           
                                        

...

 - 0s 45ms/step - loss: 0.0565 - accuracy: 0.9906 - val_loss: 0.1002 - val_accuracy: 0.9626                            

Epoch 142/200                                                                                                          
1/4 [======>.......................]                                                                                   
 - ETA: 0s - loss: 0.0558 - accuracy: 0.9792                                                                           
                                                                                                                      
4/4 [==============================]                                                                                   
 - ETA: 0s - loss: 0.0651 - accuracy: 0.9812                                                                           
                                                                                                                      
4/4 [==============================]     

                                                                                                                      
4/4 [==============================]                                                                                   
 - ETA: 0s - loss: 0.0541 - accuracy: 0.9906                                                                           
                                                                                                                      
4/4 [==============================]                                                                                   
 - 0s 51ms/step - loss: 0.0541 - accuracy: 0.9906 - val_loss: 0.0989 - val_accuracy: 0.9626                            

Epoch 157/200                                                                                                          
1/4 [======>.......................]                                                                                   
 - ETA: 0s - loss: 0.0509 - accuracy: 0.9

1/4 [======>.......................]                                                                                   
 - ETA: 0s - loss: 0.0669 - accuracy: 0.9792                                                                           
                                                                                                                      
3/4 [=====================>........]                                                                                   
 - ETA: 0s - loss: 0.0477 - accuracy: 0.9896                                                                           
                                                                                                                      
4/4 [==============================]                                                                                   
 - 0s 51ms/step - loss: 0.0462 - accuracy: 0.9906 - val_loss: 0.0980 - val_accuracy: 0.9626                            

Epoch 171/200                            

 - ETA: 0s - loss: 0.0432 - accuracy: 0.9896                                                                           
                                                                                                                      
4/4 [==============================]                                                                                   
 - ETA: 0s - loss: 0.0421 - accuracy: 0.9937                                                                           
                                                                                                                      
4/4 [==============================]                                                                                   
 - 0s 42ms/step - loss: 0.0421 - accuracy: 0.9937 - val_loss: 0.0958 - val_accuracy: 0.9626                            

Epoch 185/200                                                                                                          
1/4 [======>.......................]     

                                                                                                                      
3/4 [=====================>........]                                                                                   
 - ETA: 0s - loss: 0.0574 - accuracy: 0.9931                                                                           
                                                                                                                      
4/4 [==============================]                                                                                   
 - 0s 47ms/step - loss: 0.0559 - accuracy: 0.9937 - val_loss: 0.0935 - val_accuracy: 0.9626                            

Epoch 200/200                                                                                                          
1/4 [======>.......................]                                                                                   
 - ETA: 0s - loss: 0.0381 - accuracy: 1.0

### 6.2.1 単変量統計

In [11]:
ary=np.array([10,20,30,0])
idx=ary.argsort()
print(idx)
print(idx[::-1])

[3 0 1 2]
[2 1 0 3]


In [13]:
print(ary[idx[::-1][:3]])

[30 20 10]


In [19]:
train_y=train_y[0] #次元の調整

In [20]:
corrs=[]
for c in train_x.columns:
    corr=np.corrcoef(train_x[c],train_y)[0,1]
    corrs.append(corr)
corrs=np.array(corrs)

In [21]:
corrs

array([-0.7422521 , -0.40007457, -0.75273364, -0.72019091, -0.37580217,
       -0.6110747 , -0.71067935, -0.78344669, -0.34006406,  0.00219074,
       -0.58404735,  0.0196866 , -0.56928067, -0.54905821,  0.08258024,
       -0.30390942, -0.3010308 , -0.44692286,  0.01767304, -0.10570806,
       -0.78809673, -0.43737519, -0.7934492 , -0.74522823, -0.41532184,
       -0.59781303, -0.66400473, -0.79992591, -0.39225537, -0.31765488])

In [22]:
import scipy.stats as st

#スピアマンの順位相関係数
corrs_sp=[]
for c in train_x.columns:
    corr_sp=st.spearmanr(train_x[c],train_y).correlation
    corrs_sp.append(corr_sp)
corrs_sp=np.array(corrs_sp)

In [23]:
corrs_sp

array([-0.74190279, -0.43809031, -0.7557708 , -0.74255445, -0.36967572,
       -0.61686318, -0.73199548, -0.78205079, -0.33606184,  0.02296991,
       -0.64030767, -0.01551111, -0.65221813, -0.73299878,  0.05974938,
       -0.38504724, -0.48280689, -0.51661924,  0.10153453, -0.19448228,
       -0.79310614, -0.45411536, -0.80054358, -0.79122439, -0.40313229,
       -0.60645644, -0.70255583, -0.78551406, -0.36704372, -0.29399865])

In [24]:
#重要度の上位を出力する
idx=np.argsort(np.abs(corrs))[::-1]
top_cols,top_importances=train_x.columns.values[idx][:5],corrs[idx][:5]
print(top_cols,top_importances)

[27 22 20  7  2] [-0.79992591 -0.7934492  -0.78809673 -0.78344669 -0.75273364]


In [25]:
idx2=np.argsort(np.abs(corrs_sp))[::-1]
top_cols2,top_importances2=train_x.columns.values[idx][:5],corrs_sp[idx][:5]
print(top_cols2,top_importances2)

[27 22 20  7  2] [-0.78551406 -0.80054358 -0.79310614 -0.78205079 -0.7557708 ]


In [26]:
#順位相関係数は順位の相関係数を求めると同じ結果になる。
x=np.array([6,4,5,10,2,8,3,9,1,7])
y=np.array([10,1,4,9,3,8,6,5,2,7])

np.corrcoef(x,y)

array([[1.        , 0.67272727],
       [0.67272727, 1.        ]])

In [27]:
st.spearmanr(x,y).correlation

0.6727272727272726

In [28]:
#カイ2乗検定
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

x=MinMaxScaler().fit_transform(train_x)
c2, _=chi2(x,train_y) #2つ目はp値

#重要度の上位を出力する
idx=np.argsort(c2)[::-1]
top_cols,top_importances=train_x.columns.values[idx][:5],corrs[idx][:5]
print(top_cols,top_importances)

[ 7  6 27 23 22] [-0.78344669 -0.71067935 -0.79992591 -0.74522823 -0.7934492 ]


In [30]:
#相互情報量
from sklearn.feature_selection import mutual_info_classif

#相互情報量
mi=mutual_info_classif(train_x,train_y)

#重要度の上位を出力する
idx=np.argsort(mi)[::-1]
top_cols,top_importances=train_x.columns.values[idx][:5],corrs[idx][:5]
print(top_cols,top_importances)

[22 23 27 20  7] [-0.7934492  -0.74522823 -0.79992591 -0.78809673 -0.78344669]


### 6.2.2 特徴量の重要度を使う方法

In [32]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=10,random_state=71)
clf.fit(train_x,train_y)
fi=clf.feature_importances_

In [33]:
#重要度の上位を出力する
idx=np.argsort(fi)[::-1]
top_cols,top_importances=train_x.columns.values[idx][:5],fi[idx][:5]
print('random forest importance')
print(top_cols,top_importances)

random forest importance
[22  3  0 23 27] [0.24440855 0.14200458 0.14028729 0.09438782 0.08348286]


In [40]:
#xgboost
import xgboost as xgb
dtrain=xgb.DMatrix(train_x,label=train_y)
params={'objective':'binary:logistic','silent':1,'random_state':71}
num_round=50
model=xgb.train(params,dtrain,num_round)

#重要度の上位を出力する
fscore=model.get_score(importance_type='total_gain')
fscore=sorted([(k,v) for k,v in fscore.items()],key=lambda tpl:tpl[1],reverse=True)
print('xgboost importance')
print(fscore[:5])

[22:16:03] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


xgboost importance
[('27', 343.20337081330007), ('22', 261.09277801), ('23', 197.17255856400004), ('7', 81.12512394200002), ('13', 37.85550296499998)]
